<h1>IDA Lab Week6: Collaborative Filtering</h1>
<h2>By Smeet Dedhia Reg: 200968236</h2>

<h3>1]Summary of MovieLens Metadata:</h3>
<h4>This dataset was generated on September 26, 2018.</h4>
<h4>Format: CSV with single header row.    Encoding: UTF-8</h4><br>
1.Contains 100836 ratings and 3683 tag applications across 9742 movies<br><br>
2.Users selected at random for inclusion. All selected users had rated at least 20 movies<br><br>
3.Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).<br><br>
4.Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.<br><br>
5.Movie titles are entered manually or imported from <https://www.themoviedb.org/>, and include the year of release in parentheses. Errors and inconsistencies may exist in these titles.<br><br>
6.Genres are a pipe-separated list, and are selected from the following:
* Action* Adventure* Animation* Children's* Comedy* Crime* Documentary* Drama* Fantasy* Film-Noir* Horror* Musical* Mystery* Romance* Sci-Fi* Thriller* War* Western* (no genres listed)<br>

In [71]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

In [3]:
ratings = pd.read_csv("movielens/ratings.csv")
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [4]:
ratings.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [49]:
print("2]Pivot table: UserID vs MovieID, value = Ratings")
pd.pivot_table(ratings, index='userId', columns='movieId', values='rating')

2]Pivot table: UserID vs MovieID, value = Ratings


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df = pd.pivot_table(ratings, index='userId', columns='movieId', values='rating', fill_value=0)
df

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0,0.0,4.0,...,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0


In [50]:
print("3]Distance between all pairs of users using COSINE metric")
distance_matrix = pairwise_distances(df, metric='cosine')
distance_matrix

3]Distance between all pairs of users using COSINE metric


array([[0.00000000e+00, 9.72717135e-01, 9.40279738e-01, ...,
        7.08902628e-01, 9.06428070e-01, 8.54679193e-01],
       [9.72717135e-01, 1.11022302e-16, 1.00000000e+00, ...,
        9.53789046e-01, 9.72434599e-01, 8.97573246e-01],
       [9.40279738e-01, 1.00000000e+00, 0.00000000e+00, ...,
        9.78871538e-01, 1.00000000e+00, 9.67881252e-01],
       ...,
       [7.08902628e-01, 9.53789046e-01, 9.78871538e-01, ...,
        0.00000000e+00, 8.78007286e-01, 6.77945142e-01],
       [9.06428070e-01, 9.72434599e-01, 1.00000000e+00, ...,
        8.78007286e-01, 0.00000000e+00, 9.46774537e-01],
       [8.54679193e-01, 8.97573246e-01, 9.67881252e-01, ...,
        6.77945142e-01, 9.46774537e-01, 1.11022302e-16]])

In [61]:
print("6] 5 most similar users for user with user Id 25.")
print((-distance_matrix[24]).argsort()[1:6]+1)

6] 5 most similar users for user with user Id 25.
[161 163 546 170 175]


In [9]:
movies = pd.read_csv("movielens/movies.csv")
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [48]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [12]:
common_movies = set(ratings.loc[ratings.userId==1, 'movieId']).intersection(set(ratings.loc[ratings.userId==338, 'movieId']))
common_movies

{50, 296, 527, 593, 2959}

In [62]:
print("7]Movies, user 1 and user 338 have watched in common\n")
for movieID in sorted(common_movies):
    print(movies[movies['movieId'] == movieID])
    print('\tRating by User 1: ', ratings.loc[((ratings.userId==1) & (ratings.movieId==movieID)), 'rating'].values[0])
    print('\tRating by User 338: ', ratings.loc[((ratings.userId==338) & (ratings.movieId==movieID)), 'rating'].values[0])
    print("\n")

7]Movies, user 1 and user 338 have watched in common

    movieId                       title                  genres
46       50  Usual Suspects, The (1995)  Crime|Mystery|Thriller
	Rating by User 1:  5.0
	Rating by User 338:  4.5


     movieId                title                       genres
257      296  Pulp Fiction (1994)  Comedy|Crime|Drama|Thriller
	Rating by User 1:  3.0
	Rating by User 338:  4.5


     movieId                    title     genres
461      527  Schindler's List (1993)  Drama|War
	Rating by User 1:  5.0
	Rating by User 338:  5.0


     movieId                             title                 genres
510      593  Silence of the Lambs, The (1991)  Crime|Horror|Thriller
	Rating by User 1:  4.0
	Rating by User 338:  4.0


      movieId              title                       genres
2226     2959  Fight Club (1999)  Action|Crime|Drama|Thriller
	Rating by User 1:  5.0
	Rating by User 338:  4.5




In [26]:
common_movies2 = set(ratings.loc[((ratings.userId==2) & (ratings.rating>=4.0)), 'movieId']).intersection(set(ratings.loc[((ratings.userId==338) & (ratings.rating>=4.0)), 'movieId']))
common_movies2

{6874}

In [65]:
print("8]Common movie(s) between user 2 and user 338 with least rating of 4.0")
for movieID in sorted(common_movies2):
    df3 = movies[movies['movieId'] == movieID]
    print(df3)

8]Common movie(s) between user 2 and user 338 with least rating of 4.0
      movieId                     title                 genres
4615     6874  Kill Bill: Vol. 1 (2003)  Action|Crime|Thriller


In [66]:
print("9A]Pivot table: MovieID vs UserID, value = Ratings")
df4 = pd.pivot_table(ratings, index='movieId', columns='userId', values='rating', fill_value=0)
df4

9A]Pivot table: MovieID vs UserID, value = Ratings


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4,0.0,0.0,0,4,0,4.5,0,0,0.0,...,4.0,0,4,3,4.0,2.5,4,2.5,3,5.0
2,0,0.0,0.0,0,0,4,0.0,4,0,0.0,...,0.0,4,0,5,3.5,0.0,0,2.0,0,0.0
3,4,0.0,0.0,0,0,5,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,2.0,0,0.0
4,0,0.0,0.0,0,0,3,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,0.0,0,0.0
5,0,0.0,0.0,0,0,5,0.0,0,0,0.0,...,0.0,0,0,3,0.0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0,0.0,0.0,0,0,0,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,0.0,0,0.0
193583,0,0.0,0.0,0,0,0,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,0.0,0,0.0
193585,0,0.0,0.0,0,0,0,0.0,0,0,0.0,...,0.0,0,0,0,0.0,0.0,0,0.0,0,0.0


In [67]:
print("9B]Distance between all pairs of users using CORRELATION metric")
distance_matrix2 = pairwise_distances(df4, metric='correlation')
distance_matrix2

9B]Distance between all pairs of users using CORRELATION metric


array([[0.        , 0.7686735 , 0.8267872 , ..., 1.0289057 , 1.0289057 ,
        1.0289057 ],
       [0.7686735 , 0.        , 0.80805513, ..., 1.01829073, 1.01829073,
        1.01829073],
       [0.8267872 , 0.80805513, 0.        , ..., 1.01172923, 1.01172923,
        1.01172923],
       ...,
       [1.0289057 , 1.01829073, 1.01172923, ..., 0.        , 0.        ,
        1.00164204],
       [1.0289057 , 1.01829073, 1.01172923, ..., 0.        , 0.        ,
        1.00164204],
       [1.0289057 , 1.01829073, 1.01172923, ..., 1.00164204, 1.00164204,
        0.        ]])

In [37]:
for movie in movies.title:
    if('Godfather' in str(movie)):
        print(movie)

Godfather, The (1972)
Godfather: Part II, The (1974)
Godfather: Part III, The (1990)
Tokyo Godfathers (2003)
The Godfather Trilogy: 1972-1990 (1992)


In [38]:
movies.loc[(movies.title=='Godfather, The (1972)'), 'movieId'].values[0]

858

In [40]:
similar_movies = distance_matrix2[857].argsort()[1:6]+1
similar_movies

array([2763, 2757, 1475, 2247, 1803], dtype=int64)

In [70]:
print("10]Top 5 movies which are similar to the movie “Godfather”\n")
for movieID in similar_movies:
    print(movies[movies["movieId"] == movieID])
    print("\n")

10]Top 5 movies which are similar to the movie “Godfather”

      movieId                            title          genres
2079     2763  Thomas Crown Affair, The (1999)  Action|Mystery


      movieId           title genres
2075     2757  Frances (1982)  Drama


      movieId                              title   genres
1128     1475  Kama Sutra: A Tale of Love (1996)  Romance


      movieId                      title  genres
1670     2247  Married to the Mob (1988)  Comedy


Empty DataFrame
Columns: [movieId, title, genres]
Index: []




In the above question, it is seen that data for Movie ID 1803 is not present in the movies.csv dataset, hence its details cannot be printed.